In [1]:
import gensim
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
#le data frame est saved à partir de Help_rapport 
data_cleaned = pd.read_csv("data/data_cleaned_1M.csv").fillna("")
data_cleaned

,Categorie1,Description
0,INFORMATIQUE,batter acer aspir one yr li ion mah wh noir co...
1,TELEPHONIE - GPS,coqu rigid bleu lagon pour alcatel ot motif dr...
2,TELEPHONIE - GPS,facad et coqu cellular lin shckgal minip marqu...
3,TELEPHONIE - GPS,coqu meteor tpu lg nexus
4,TELEPHONIE - GPS,coqu soupl transparent pour lg flex motif keep...
...,...,...
999995,BRICOLAGE - OUTILLAGE - QUINCAILLERIE,vis tet cylindr toolcraft din aci mm pc conten...
999996,INSTRUMENTS DE MUSIQUE,cord thomastik violon domin noyau plein nylon ...
999997,ELECTROMENAGER,puissanc pression maximal bar reservoir amovib...
999998,VETEMENTS - LINGERIE,gilet hop lif yedo noir nouvel collect de la l...


In [3]:
## On s'est rendu compte que le dernier token était "" pour chaque produit
train_array_token = [line.split(' ') for line in data_cleaned.Description.values]
print(train_array_token[:5])

[['batter', 'acer', 'aspir', 'one', 'yr', 'li', 'ion', 'mah', 'wh', 'noir', 'compatibl', 'batter', 'la'], ['coqu', 'rigid', 'bleu', 'lagon', 'pour', 'alcatel', 'ot', 'motif', 'drapeau', 'liberi', 'film', 'coqu', 'rigid', 'ultra', 'fin', 'bleu', 'lagon', 'original', 'de', 'muzzano', 'au', 'motif', 'drapeau', 'liberi', 'pour', 'alcatel', 'la'], ['facad', 'et', 'coqu', 'cellular', 'lin', 'shckgal', 'minip', 'marqu', 'agree', 'samsungmobil', 'compatibl', 'galaxy', 'minimatier', 'caoutchouc', 'soupl', 'la'], ['coqu', 'meteor', 'tpu', 'lg', 'nexus'], ['coqu', 'soupl', 'transparent', 'pour', 'lg', 'flex', 'motif', 'keep', 'calm', 'and', 'play', 'football', 'coqu', 'soupl', 'ultra', 'fin', 'transparent', 'original', 'de', 'muzzano', 'au', 'motif', 'keep', 'calm', 'and', 'play', 'footbal', 'la']]


In [4]:
Features_dimension = 500
hs = 0
negative = 10

In [5]:
sg = 1
print("Start learning skip-gram Word2Vec")
ts = time.time()
model_sg = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)

Start learning skip-gram Word2Vec
Learning time : 699.03 seconds Word2Vec


In [6]:
import numpy as np

In [7]:
#fonction qui crée la list de features

def set_features(array_token_prod,dim_feature,model):
    features= []
    #on prend les tokens de chaque produit
    for token_prod in array_token_prod:
        #on crée le np final du produit
        feat_prod= np.zeros(dim_feature)
        # on prend chaque token dans la desc prod
        for token in token_prod:
            #on somme les vecteurs et on divise direct par le nombre de token
            feat_prod +=model[token]/len(token_prod)
            
        #on rajoute le vectuer moyenné produit dans la list features
        features += [feat_prod]
    return features

In [8]:
#init notre list de features
features=set_features(train_array_token,Features_dimension,model_sg)


/Users/paulbrunet/anaconda3/envs/Projet4A/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [ ]:
len(features) 

On a bien 100 000 vecteurs colonne de dim 300

In [9]:
#on va split comme précédemment avec ces nvx features et label
label = data_cleaned["Categorie1"].values
X_train, X_test, y_train, y_test = train_test_split(features,label, test_size=0.1, random_state=42)


Procédure habituelle ensuite..

In [10]:
ts = time.time()
lr = LogisticRegression()
lr.fit(X_train, y_train)
te = time.time()
print(te-ts, "sec")

/Users/paulbrunet/anaconda3/envs/Projet4A/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


789.879961013794 sec


In [11]:
print("Score apprentissage ",lr.score(X_train,y_train))
print("Score test ",lr.score(X_test,y_test))

Score apprentissage  0.9030055555555555
Score test  0.90134


### Autre modèle:

In [ ]:
sg = 0
print("Start learning CBOW Word2Vec")
ts = time.time()
model_cbow = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)

In [ ]:
features2=set_features(train_array_token,Features_dimension,model_cbow)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2,label, test_size=0.1, random_state=42)

In [ ]:
ts = time.time()
lr2 = LogisticRegression()
lr2.fit(X_train2, y_train2)
te = time.time()
print(te-ts, "sec")

In [ ]:
print("Score apprentissage ",lr2.score(X_train2,y_train2))
print("Score test ",lr2.score(X_test2,y_test2))

## Exploration utilisation modèle/token

In [ ]:
model_sg.most_similar(["montr"])

In [ ]:
model_cbow.most_similar(["montr"])

In [ ]:
model_sg["montr"].shape

In [ ]:
#faire tourner sg sur le max de données avec données nettoyées
#puis prediction avec la regression à comparer le mm modele avec les features de count vectorizer

# Extra Work

pre-trained network


Github du projet[https://github.com/Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors) appris sur 1Giga d'articles de wikipedia en mode **Skip-Gram*

Vous pouvez télécharger ce modèle en suivant ce [lien](https://drive.google.com/file/d/0B0ZXk88koS2KM0pVTktxdG15TkE/view). Dezipez-le puis téléchargez le modèle en indiquant la direction du fichier "fr/bin"

In [ ]:
model_online_dir = "data/fr/fr.bin"
#model_online_dir = "ACOMPLETER/fr.bin"
model_online = gensim.models.Word2Vec.load(model_online_dir)